In [4]:
# 모듈 import
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
import wandb
import copy

In [5]:
import torch
torch.cuda.empty_cache()

In [6]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./DyHead/swin_s_swin_t_fpn_dyhead_1x_coco.py')
root='/opt/ml/detection/dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = './stratified_kfold/cv_train_1.json' 
# cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.val.classes = classes
cfg.data.val.img_prefix = root
cfg.data.val.ann_file = './stratified_kfold/cv_val_1.json'
# cfg.data.val.pipeline[1]['img_scale'] = (1024,1024) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json'
# cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 4

cfg.seed = 2021
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/atss_swin_t_fpn_dyhead_1x'

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
# cfg.runner.max_epochs = 100
cfg.checkpoint_config = dict(max_keep_ckpts=1, interval=1)

In [7]:
# build_dataset
# datasets = [build_dataset(cfg.data.train),build_dataset(cfg.data.val)]
datasets = [build_dataset(cfg.data.train)]
datasets

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!


[
 CocoDataset Train dataset with number of images 3928, and instance counts: 
 +-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
 | category          | count | category      | count | category        | count | category    | count | category     | count |
 +-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
 | 0 [General trash] | 3154  | 1 [Paper]     | 5076  | 2 [Paper pack]  | 741   | 3 [Metal]   | 780   | 4 [Glass]    | 752   |
 | 5 [Plastic]       | 2331  | 6 [Styrofoam] | 1028  | 7 [Plastic bag] | 4125  | 8 [Battery] | 121   | 9 [Clothing] | 396   |
 +-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+]

In [ ]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

In [9]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

2022-03-30 01:29:58,530 - mmdet - INFO - Start running, host: root@6f46492e33de, work_dir: /opt/ml/ysw/level2-object-detection-level2-cv-18/Mmdetection/_boost_/work_dirs/atss_swin_t_fpn_dyhead_1x
2022-03-30 01:29:58,532 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) CosineRestartLrUpdaterHook         
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) CosineRestartLrUpdaterHook         
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_iter:
(VERY_HIGH   ) CosineRestartLrUpdaterHook         
(LOW   

loading annotations into memory...
Done (t=0.17s)
creating index...
index created!


/opt/conda/envs/detection/lib/python3.7/site-packages/mmcv/runner/hooks/evaluation.py:222: UserWarning: runner.meta is None. Creating an empty one.
  warnings.warn('runner.meta is None. Creating an empty one.')
wandb: Currently logged in as: ysw2946 (use `wandb login --relogin` to force relogin)


2022-03-30 01:31:59,488 - mmdet - INFO - Epoch [1][50/982]	lr: 4.995e-06, eta: 21:56:38, time: 2.238, data_time: 0.070, memory: 22141, loss_rpn_cls: 0.6855, loss_rpn_bbox: 0.0114, s0.loss_cls: 1.8105, s0.acc: 98.6650, s0.loss_bbox: 0.0118, s1.loss_cls: 1.0162, s1.acc: 77.1211, s1.loss_bbox: 0.0040, s2.loss_cls: 0.4872, s2.acc: 96.2168, s2.loss_bbox: 0.0006, loss: 4.0271, grad_norm: 10.6458
2022-03-30 01:33:49,669 - mmdet - INFO - Epoch [1][100/982]	lr: 9.990e-06, eta: 21:44:43, time: 2.204, data_time: 0.020, memory: 22141, loss_rpn_cls: 0.6195, loss_rpn_bbox: 0.0123, s0.loss_cls: 0.2667, s0.acc: 98.2568, s0.loss_bbox: 0.0248, s1.loss_cls: 0.1711, s1.acc: 98.8564, s1.loss_bbox: 0.0068, s2.loss_cls: 0.0746, s2.acc: 99.0684, s2.loss_bbox: 0.0011, loss: 1.1770, grad_norm: 5.5083
2022-03-30 01:35:35,596 - mmdet - INFO - Epoch [1][150/982]	lr: 1.499e-05, eta: 21:22:53, time: 2.119, data_time: 0.016, memory: 22141, loss_rpn_cls: 0.4374, loss_rpn_bbox: 0.0142, s0.loss_cls: 0.1269, s0.acc: 97.5

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 955/955, 3.1 task/s, elapsed: 304s, ETA:     0s

...

Loading and preparing results...
DONE (t=0.03s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=2.06s).
Accumulating evaluation results...


2022-03-30 08:52:26,978 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.004
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.011
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.002
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.005
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.032
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.032
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.032
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=0.72s).


2022-03-30 08:52:37,846 - mmdet - INFO - Now best checkpoint is saved as best_bbox_mAP_epoch_12.pth.
2022-03-30 08:52:37,849 - mmdet - INFO - Best bbox_mAP is 0.0040 at 12 epoch.
2022-03-30 08:52:37,856 - mmdet - INFO - Epoch(val) [12][955]	bbox_mAP: 0.0040, bbox_mAP_50: 0.0110, bbox_mAP_75: 0.0020, bbox_mAP_s: 0.0000, bbox_mAP_m: 0.0000, bbox_mAP_l: 0.0050, bbox_mAP_copypaste: 0.004 0.011 0.002 0.000 0.000 0.005
2022-03-30 08:54:32,489 - mmdet - INFO - Epoch [13][50/982]	lr: 1.153e-05, eta: 14:02:30, time: 2.292, data_time: 0.069, memory: 22141, loss_rpn_cls: 0.0352, loss_rpn_bbox: 0.0142, s0.loss_cls: 0.1368, s0.acc: 96.7793, s0.loss_bbox: 0.0604, s1.loss_cls: 0.0467, s1.acc: 97.8809, s1.loss_bbox: 0.0287, s2.loss_cls: 0.0143, s2.acc: 98.7305, s2.loss_bbox: 0.0057, loss: 0.3422, grad_norm: 0.9076
2022-03-30 08:56:24,066 - mmdet - INFO - Epoch [13][100/982]	lr: 1.153e-05, eta: 14:00:51, time: 2.232, data_time: 0.022, memory: 22141, loss_rpn_cls: 0.0343, loss_rpn_bbox: 0.0107, s0.loss_

KeyboardInterrupt: 

In [ ]:
import torch
torch.cuda.empty_cache()